In [1]:
import torch
import os
import pandas as pd
from transformers import AutoFeatureExtractor
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
# from fairlearn.metrics import MetricFrame, demographic_parity_difference, equalized_odds_difference
# import wandb
import matplotlib.pyplot as plt
# import seaborn as sns
from transformers import AutoModelForAudioClassification
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from utils.dataset_utils import CremaDataset

In [4]:
# pip3 install fairlearn
# pip3 install tqdm
# pip3 install seaborn
# pip3 install wandb
# pip3 install datasets
# pip3 install transformers
# pip3 install librosa

In [6]:
wandb.init(project="crema_evaluation_with_fairness", name="evaluation_run")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: axhero7 (axhero7-organization). Use `wandb login --relogin` to force relogin


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
dataset = load_dataset("myleslinder/crema-d", trust_remote_code=True, split='train')
id2label = {'0': 'neutral', '1': 'happy', '2': 'sad', '3': 'anger', '4': 'fear', '5': 'disgust'}
label2id = {v: k for k, v in id2label.items()}

# Select necessary columns
dataset = dataset.select_columns(['audio', 'label'])
dataset = dataset.train_test_split(test_size=0.3)

model_id = "distil-whisper/distil-medium.en"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id, do_normalize=True)

In [2]:
model_id = "distil-whisper/distil-medium.en"
dataset = CremaDataset(model_id)
dataset.set_vector("actor-vector.hf", True)

TypeError: CremaDataset.__init__() missing 1 required positional argument: 'model_id'

In [9]:
demographics_path = "VideoDemographics.csv"
demographics = pd.read_csv(demographics_path)
demographics.head()

,ActorID,Age,Sex,Race,Ethnicity
0,1001,51,Male,Caucasian,Not Hispanic
1,1002,21,Female,Caucasian,Not Hispanic
2,1003,21,Female,Caucasian,Not Hispanic
3,1004,42,Female,Caucasian,Not Hispanic
4,1005,29,Male,African American,Not Hispanic


In [31]:
BATCH_SIZE = 4

# test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE)


model = AutoModelForAudioClassification.from_pretrained(model_id, num_labels=len(id2label))
model.to(device)

# Load pre-trained model weights
model.load_state_dict(torch.load("model.pth"))

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at distil-whisper/distil-medium.en and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [27]:
def make_prediction(model, batch, device):
    inputs = {key: val.to(device) for key, val in batch.items()}
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, dim=-1)
    return preds

In [63]:
# for batch in tqdm(dataset['train'], desc="Evaluating"):
for batch in range(0, len(dataset.test_dataset)):
    print(dataset.test_dataset[batch])
    inputs = {"input_features": dataset.test_dataset[batch]["input_features"].unsqueeze(0).to(device)}
    labels = dataset.test_dataset["labels"]
    print(inputs)
    outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, dim=-1)
    print(preds)
    print(labels)
    preds = make_prediction(model, batch, device)
    
   
    break

{'labels': tensor(4), 'input_features': tensor([[ 0.3249,  0.2401,  0.3034,  ..., -1.0937, -1.0937, -1.0937],
        [ 0.3307,  0.2953,  0.2271,  ..., -1.0937, -1.0937, -1.0937],
        [ 0.3051,  0.3750,  0.0698,  ..., -1.0937, -1.0937, -1.0937],
        ...,
        [-0.5717, -0.6035, -0.5495,  ..., -1.0937, -1.0937, -1.0937],
        [-0.4953, -0.5553, -0.4541,  ..., -1.0937, -1.0937, -1.0937],
        [-0.5559, -0.5777, -0.5451,  ..., -1.0937, -1.0937, -1.0937]]), 'path': '1061'}
{'input_features': tensor([[[ 0.3249,  0.2401,  0.3034,  ..., -1.0937, -1.0937, -1.0937],
         [ 0.3307,  0.2953,  0.2271,  ..., -1.0937, -1.0937, -1.0937],
         [ 0.3051,  0.3750,  0.0698,  ..., -1.0937, -1.0937, -1.0937],
         ...,
         [-0.5717, -0.6035, -0.5495,  ..., -1.0937, -1.0937, -1.0937],
         [-0.4953, -0.5553, -0.4541,  ..., -1.0937, -1.0937, -1.0937],
         [-0.5559, -0.5777, -0.5451,  ..., -1.0937, -1.0937, -1.0937]]],
       device='cuda:0')}
tensor([5], device='cud

NameError: name 'make_prediction' is not defined

In [33]:
dataset['test'][0]['audio']['path'].split('/')[-1][:4]

'1059'

In [53]:
dataset['test']

{'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/a5b997a2046c58b60a966d6760b86c2830b98882fce95b7c3a0658d7aa4bd7d5/data/AudioWAV/1059_IOM_FEA_XX.wav',
   'array': array([-0.00552368, -0.00549316, -0.00460815, ...,  0.        ,
           0.        ,  0.        ]),
   'sampling_rate': 16000},
  {'path': '/root/.cache/huggingface/datasets/downloads/extracted/a5b997a2046c58b60a966d6760b86c2830b98882fce95b7c3a0658d7aa4bd7d5/data/AudioWAV/1035_TIE_SAD_XX.wav',
   'array': array([0.00585938, 0.0055542 , 0.00491333, ..., 0.        , 0.        ,
          0.        ]),
   'sampling_rate': 16000}],
 'label': [4, 2]}

In [46]:
b = next(iter(test_dataloader))

NameError: name 'test_dataloader' is not defined

In [64]:
inputs = {key: val.to(device) for key, val in b.items()}
labels = inputs.pop("labels")

In [66]:
labels

tensor([3, 4, 5, 5], device='cuda:0')

In [125]:
inputs["input_features"].shape

torch.Size([4, 80, 3000])

In [90]:
test_dataset['input_features'][0].shape

torch.Size([80, 3000])

In [89]:
for i in range(0, len(test_dataset)):
    print(torch.tensor(dataset['test']['audio'][i]['array']))
    break

tensor([-0.0055, -0.0055, -0.0046,  ...,  0.0000,  0.0000,  0.0000],
       dtype=torch.float64)
